In [1]:
import os 
path = "/home/ruthvik1994/Grive/01.ALDA/Project-Alda/data"
from pyspark.mllib.recommendation import ALS
import math

In [3]:
ratings = sc.textFile(os.path.join(path, 'ratings.csv')) \
    .map(lambda line: line.split(",")) \
    .filter(lambda line: len(line)>1) \
    .map(lambda a: (a[0], a[1], a[2]))

header = ratings.first()
ratings = ratings.filter(lambda line: line != header)
ratings = ratings.map(lambda x:(int(x[0]), x[1], x[2]))
print ("Total number of ratings : %d"%ratings.count())

Total number of ratings : 100004


In [5]:
movies = sc.textFile(os.path.join(path, 'movies.csv'))\
        .map(lambda line:line.split(","))\
        .filter(lambda line:len(line)>1)\
        .map(lambda a:(a[0],a[1]))
header2 = movies.first()
movies = movies.filter(lambda line: line != header2)
print ("Total number of Movies : %d"%movies.count())

Total number of Movies : 9125
